# Circuit Composition
We provide some utility functions to join, slice, and otherwise manipulate quantum circuits. The quick-reference guide is:
 * `copy` - make a copy of the circuit. Setting the params of the new circuit will not affect the params of the original circuit.
 * `join_in_qubits` - join a list of circuits in qubit space.
 * `join_in_time` - join a list of circuits in time.
 * `slice` - slice out a subset of times and/or qubits.
 * `reverse` - return a reversed-in-time version.
 * `adjoint` - return the adjoint of the circuit.
 * `sparse` - remove empty time and/or qubit moments.
 * `center` - translate the circuit to a new origin.
 * `add_gate` - an explicit method to add individual gates at specified qubits and times.
 * `add_gates` - add the gates another circuit into self, essentially a sequential version of `add_gate`.
 * `remove_gate` - remove a gate at a given qubits/times key.
 * `replace_gate` - replace a gate at a given qubits/times key.

## Slicing and Dicing

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('circuit_composition.ipynb')
import numpy as np
import quasar

Use `join_in_qubits` static method to join two or more circuits in qubit space:

In [2]:
circuit_ry = quasar.Circuit()
circuit_ry.Ry(0)
print(circuit_ry)

T  : |0 |

q0 : -Ry-
         
T  : |0 |



In [3]:
circuit_ry2 = quasar.Circuit.join_in_qubits([circuit_ry]*2)
print(circuit_ry2)

T  : |0 |

q0 : -Ry-
         
q1 : -Ry-
         
T  : |0 |



Use the `join_in_time` static method to join two or more circuits in time:

In [4]:
circuit_cx = quasar.Circuit()
circuit_cx.CX(0,1)
print(circuit_cx)

T  : |0|

q0 : -@-
      | 
q1 : -X-
        
T  : |0|



In [5]:
circuit = quasar.Circuit.join_in_time([circuit_ry2, circuit_cx]*2 + [circuit_ry2])
print(circuit)

T  : |0 |1|2 |3|4 |

q0 : -Ry-@-Ry-@-Ry-
         |    |    
q1 : -Ry-X-Ry-X-Ry-
                   
T  : |0 |1|2 |3|4 |



Use the `slice` method to extract a slice of a subset of time values by supplying the `times` argument:

In [6]:
print(circuit.slice(times=[0,1,2]))

T  : |0 |1|2 |

q0 : -Ry-@-Ry-
         |    
q1 : -Ry-X-Ry-
              
T  : |0 |1|2 |



Use the `slice` method to extract a subset of qubit indices by supplying the `qubits` argument:

In [7]:
print(circuit.slice(qubits=[1]))

T  : |0 |1|2 |3|4 |

q0 : -Ry---Ry---Ry-
                   
T  : |0 |1|2 |3|4 |



Use the `sparse` method to remove empty time and/or qubit indices:

In [8]:
print(circuit.slice(qubits=[1]).sparse())

T  : |0 |1 |2 |

q0 : -Ry-Ry-Ry-
               
T  : |0 |1 |2 |



Use the `reverse` method to obtain the time-reversed ordering of the circuit (gate time order reversed, but no gate adjoints taken):

In [9]:
print(circuit.reverse())

T  : |0 |1|2 |3|4 |

q0 : -Ry-@-Ry-@-Ry-
         |    |    
q1 : -Ry-X-Ry-X-Ry-
                   
T  : |0 |1|2 |3|4 |



Use the `adjoint` method to obtain the adjoint of the circuit (gate time order reversed and gate adjoints taken):

In [10]:
print(circuit.adjoint())

T  : |0 |1|2 |3|4 |

q0 : -Ry-@-Ry-@-Ry-
         |    |    
q1 : -Ry-X-Ry-X-Ry-
                   
T  : |0 |1|2 |3|4 |



## Slice: Another Look

The `slice` method is actually rather powerful due to the use of optional arguments for `qubits` and `times` (source keys in the circuit being sliced) and `qubits_to` and `times_to` (target keys in the circuit being constructed by the slice). We will examine these rules on the following example circuit:

In [11]:
circuit = quasar.Circuit()
circuit.X(0).Y(1).Z(3)
circuit.Y(0).Z(1).X(3)
circuit.Z(0, times=3).X(1, times=3).Y(3, times=3)
print(circuit)

T  : |0|1|2|3|

q0 : -X-Y---Z-
              
q1 : -Y-Z---X-
              
q2 : ---------
              
q3 : -Z-X---Y-
              
T  : |0|1|2|3|



By default, if either the `times` or `qubits` arguments is not specified, that dimension of the circuit is left untouched. If the `times` or `qubits` argument is specified but the corresponding `times_to` or `qubits_to` argument is not specified, the `_to` target indices are inferred to start at zero and increase sequentially. E.g., the following call to `slice` maps `qubits=[0, 1, 3]` to `qubits_to=[0, 1, 2]`, and leaves the time keys untouched:

In [12]:
print(circuit.slice(qubits=[0, 1, 3]))

T  : |0|1|2|3|

q0 : -X-Y---Z-
              
q1 : -Y-Z---X-
              
q2 : -Z-X---Y-
              
T  : |0|1|2|3|



Similarly, the following call to slice maps `times=[0, 1, 3]` to `times_to=[0, 1, 2]` and leaves the qubit keys untouched:

In [13]:
print(circuit.slice(times=[0, 1, 3]))

T  : |0|1|2|

q0 : -X-Y-Z-
            
q1 : -Y-Z-X-
            
q2 : -------
            
q3 : -Z-X-Y-
            
T  : |0|1|2|



Providing both source and target indices to `slice` allows for more explicit control, e.g., by reordering time and/or qubit keys while controlling empty spaces in the circuit:

In [14]:
print(circuit.slice(qubits=[0, 1, 3], qubits_to=[3, 1, 0]))

T  : |0|1|2|3|

q0 : -Z-X---Y-
              
q1 : -Y-Z---X-
              
q2 : ---------
              
q3 : -X-Y---Z-
              
T  : |0|1|2|3|



In [15]:
print(circuit.slice(times=[0, 1, 3], times_to=[5, 1, 0]))

T  : |0|1|2|3|4|5|

q0 : -Z-Y-------X-
                  
q1 : -X-Z-------Y-
                  
q2 : -------------
                  
q3 : -Y-X-------Z-
                  
T  : |0|1|2|3|4|5|



In all cases, the qubits and times arguments can be freely mixed:

In [16]:
print(circuit.slice(
    qubits=[0, 1],
    qubits_to=[-2, -1],
    times=[0, 1, 3], 
    times_to=[5, 1, 0]))

T   : |0|1|2|3|4|5|

q-2 : -Z-Y-------X-
                   
q-1 : -X-Z-------Y-
                   
T   : |0|1|2|3|4|5|



After extensive slicing, a call to `center` can be useful to reset the origin of the circuit:

In [17]:
print(circuit.slice(
    qubits=[0, 1],
    qubits_to=[-2, -1],
    times=[0, 1, 3], 
    times_to=[5, 1, 0]).center())

T  : |0|1|2|3|4|5|

q0 : -Z-Y-------X-
                  
q1 : -X-Z-------Y-
                  
T  : |0|1|2|3|4|5|



## Fine-Grained Control: add_gate and add_gates

Fine-grained control of circuit construction can be accomplished with the `add_gate` and `add_gates` methods, and corresponding functionality in gate construction helper methods like `H`, `CX`, and `Ry`. For instance:

In [18]:
gadget = quasar.Circuit().Ry(1).CZ(0,1).Ry(1).CX(1,0)
circuit = quasar.Circuit()
circuit.Ry(qubit=0, times=0)
circuit.add_gates(gadget, (0,1), times=(0,1,2,3))
circuit.add_gates(gadget, (1,2), time_start=4)
circuit.add_gates(gadget, (2,3), time_placement='early')
circuit.add_gates(gadget, (3,4))
circuit.H(0, time_placement='next')
circuit.H(1, time_placement='late')
circuit.H(2, time_placement='late')
circuit.H(3, time_placement='late')
circuit.H(4, time_placement='late')
print(circuit)

T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|

q0 : -Ry-@----X-----------------------------H--
         |    |                                
q1 : -Ry-Z-Ry-@----@----X-------------------H--
                   |    |                      
q2 : -----------Ry-Z-Ry-@--@----X-----------H--
                           |    |              
q3 : -------------------Ry-Z-Ry-@--@-----X--H--
                                   |     |     
q4 : ---------------------------Ry-Z--Ry-@--H--
                                               
T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|



This example deserves a moment of study. First, note that all of the `Circuit` helper methods to add individual gates, such as `H`, `CX`, and `Ry` actually call the `add_gate` method, and pass along any keyword arguments. Therefore, these methods all act like the one-gate equivalent of `add_circuit`. Next, both `add_gate` and `add_circuit` always require the user to provide the `qubits` argument (either an `int` qubit index for a one-qubit gate or a `tuple` of `int` for one-or-more-qubit gates). For the helper methods such as `H`, `CX`, and `Ry`, an explicit list of qubit indices corresponding to, e.g., control (`qubitA`) and target (`qubitB`) qubits is used. For the time axis, the `add_gate` and `add_circuit` methods default to using the `'early'` `time_placement` argument, which adds the new gate or circuit at the right end of the circuit, and then slides the new gate or circuit as far to the left as possible. Optionally, the user can manually specify the `time_placement` argument as being `'early'`, `'late'`, or `'next'`. The `'late'` option tries to start adding the new gate or circuit in the last time moment present in the current circuit, but pushes the entry back to a new time moment if a conflict arises. The `'next'` option always opens a new time moment and starts entry from that point. The `time_placement` argument is trumped by the explicit `time` argument (an `int`), which always starts the entry of the new gate or circuit at the specfied `time_start`, and proceeds contiguously, with an error thrown if conflicts are found. For the `add_circuit` method, the `time_start` argument is trumped by the explicit `times` argument (a sequence of `int`) that explicitly specifies the logical mapping of added circuit time moments to the absolute time moments of `self`.

## Gate Addition Keys

When using automatic time placement features like the `time_placement` argument in `add_gate`, the user might want to keep track of where the gate was actually placed. To this end, we provide the `return_key` optional argument of `add_gate` which, when set to `True` changes the return type of `add_gate` to the qubit and time keys of the call to `add_gate` instead of the usual `self` for chaining. The `return_key` optional argument falls through all gate addition sugar methods: 

In [19]:
circuit2 = quasar.Circuit().H(0).Y(1).CX(0,1)
times, qubits = circuit2.Ry(1, theta=0.1, return_key=True)
print(circuit2)
print(times, qubits)

T  : |0|1|2 |

q0 : -H-@----
        |    
q1 : -Y-X-Ry-
             
T  : |0|1|2 |

(2,) (1,)


E.g., the last `Ry` gate above was added at qubit index `1` and time index `2`. 

## Gate Removal and Replacement

Gates can be removed from the circuit by calling the `remove_gate` method:

In [20]:
circuit.remove_gate(qubits=(4,3), times=(13,))
print(circuit)

T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|

q0 : -Ry-@----X-----------------------------H--
         |    |                                
q1 : -Ry-Z-Ry-@----@----X-------------------H--
                   |    |                      
q2 : -----------Ry-Z-Ry-@--@----X-----------H--
                           |    |              
q3 : -------------------Ry-Z-Ry-@--@--------H--
                                   |           
q4 : ---------------------------Ry-Z--Ry----H--
                                               
T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|



Note that while we guarantee the validity of `remove_gate`, we have made zero effort to make this operation efficient - there is a roughly O(`ngate`) overhead to rebuild the `Circuit` indexing arrays in each call to `remove_gate`. If, instead, your use case is to replace one gate with another, you can use the much more efficient `replace_gate` method: 

In [21]:
circuit.replace_gate(gate=quasar.Gate.CY, qubits=(3,4), times=(11,))
print(circuit)

T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|

q0 : -Ry-@----X-----------------------------H--
         |    |                                
q1 : -Ry-Z-Ry-@----@----X-------------------H--
                   |    |                      
q2 : -----------Ry-Z-Ry-@--@----X-----------H--
                           |    |              
q3 : -------------------Ry-Z-Ry-@--@--------H--
                                   |           
q4 : ---------------------------Ry-Y--Ry----H--
                                               
T  : |0 |1|2 |3|4 |5|6 |7 |8|9 |10|11|12|13|14|

